In [1]:
%matplotlib inline
%run LocalRepo.ipynb
%run repos.ipynb
%run parsing.ipynb
%run metrics.ipynb

In [2]:
repos

['jenkinsci/jenkins']

In [3]:
import_query = JA_LANGUAGE.query("(import_declaration (scoped_identifier) @decl )")
def import_node_to_list(node):
    if node.type == "scoped_identifier":
        list = import_node_to_list

In [4]:
for repo in repos:
    r = LocalRepo(repo)
    r.update()
    
    # structural = MetricsGeneration(r).calculate_structural_connections(r)
    ling = MetricsGeneration(r).calculate_linguistic_connections()
    continue
    
    found_nodes = RepoTree("")
    files = r.get_file_objects()
    print("Analyzing " + str(len(files)) + " files...")
    for file_obj in files:
        file = RepoFile(r, file_obj)
        imports = import_query.captures(file.get_tree().root_node)
        for import_statement in imports:
            print(file.node_text(import_statement[0]).split("."))
        print("-")
        def handle(logic_path, node):
            found_nodes.register(logic_path)
        file.walk_tree(handle)
    
    print("Found " + str(found_nodes.node_count()) + " classes, methods and fields!")
    
    with open("../debug-tree.json", "w") as outfile:
        outfile.write(found_nodes.to_json())
print("Everything up to date!")

updating jenkinsci/jenkins...
Done!
Extracting words...
Found words: 7062
Creating vectorizer...
Training LDA...
Generating result output...
Topic #0: inc validator registry limit domain service ltd tld valid code
Topic #1: map fingerprint xml list util name converter stream load range
Topic #2: build project run action queue listener item task job scm
Topic #3: stream exception output java key input secret write channel link
Topic #4: exception mock org protocol handler mockito java proxy jenkins socket
Topic #5: test org rule equal jenkins junit jvnet parameter foo issue
Topic #6: descriptor list extension stapler object method link property extend provider
Topic #7: user security jenkins realm property exception request servlet org crumb
Topic #8: log logger exception time java util thread event level task
Topic #9: update plugin jenkins version site json url install file job
Topic #10: text annotate calendar console cron tab test markup check exception
Topic #11: list name java uti